In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader


df=pd.read_csv("train_data.csv")
Y_train=df['Patv']
df1=pd.read_csv("val_data.csv")
Y_val=df1['Patv']
df2=pd.read_csv("test_data.csv")
Y_test=df2['Patv']
X2_train=df.drop(['Patv'],axis=1)
X2_val=df1.drop(['Patv'],axis=1)
X2_test=df2.drop(['Patv'],axis=1)

In [3]:
X2_train_series = X2_train.values.reshape((X2_train.shape[0], X2_train.shape[1], 1))
X2_valid_series = X2_val.values.reshape((X2_val.shape[0], X2_val.shape[1], 1))

subsequences = 3
timesteps = X2_train.shape[1]//subsequences
X2_train_series_sub = X2_train_series.reshape((X2_train_series.shape[0], subsequences, timesteps, 1))
X2_valid_series_sub = X2_valid_series.reshape((X2_valid_series.shape[0], subsequences, timesteps, 1))
print('Train set shape', X2_train_series_sub.shape)
print('Validation set shape', X2_valid_series_sub.shape)

Train set shape (2836512, 3, 8, 1)
Validation set shape (945504, 3, 8, 1)


In [6]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

model_cnn_lstm = Sequential()
model_cnn_lstm.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, X2_train_series_sub.shape[2], X2_train_series_sub.shape[3])))
model_cnn_lstm.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model_cnn_lstm.add(TimeDistributed(Flatten()))
model_cnn_lstm.add(LSTM(50, activation='relu'))
model_cnn_lstm.add(Dense(1))
model_cnn_lstm.compile(loss='mse', optimizer='adam')

Metal device set to: Apple M1


In [ ]:
cnn_lstm_history = model_cnn_lstm.fit(X2_train_series_sub, Y_train, 
                                      validation_data=(X2_valid_series_sub, Y_val), epochs=3, verbose=2)

In [ ]:
cnn_lstm_train_pred = model_cnn_lstm.predict(X2_train_series_sub)
cnn_lstm_valid_pred = model_cnn_lstm.predict(X2_valid_series_sub)
print('Train rmse:', np.sqrt(mean_squared_error(Y_train, cnn_lstm_train_pred)))
print('Validation rmse:', np.sqrt(mean_squared_error(Y_val, cnn_lstm_valid_pred)))